# PROYECTO TOMA DE DESICIONES A/A/B TEST


## Introducción

En el entorno altamente competitivo de las aplicaciones móviles de comercio electrónico, comprender el comportamiento del usuario es clave para mejorar la conversión y optimizar la experiencia digital. Este proyecto se desarrolla en el contexto de una empresa emergente dedicada a la venta de productos alimenticios a través de una aplicación móvil. El equipo de diseño propone un cambio en la tipografía de la aplicación para mejorar su apariencia, pero existe preocupación sobre cómo este cambio puede afectar la percepción y la interacción del usuario. Para tomar decisiones fundamentadas, se ha planteado un análisis exhaustivo de los datos de uso de la aplicación, con especial énfasis en el embudo de ventas y en los resultados de un experimento controlado tipo A/A/B.

## Objetivo General
Analizar el comportamiento de los usuarios dentro de la aplicación de una empresa emergente de productos alimenticios mediante el estudio del embudo de ventas y la evaluación de un experimento A/A/B, con el fin de identificar cuellos de botella en el proceso de conversión y determinar si un cambio en la tipografía tiene un impacto significativo en la interacción de los usuarios.

## Descripción de los datos
Cada entrada de registro es una acción de usuario o un evento.

- ``EventName``: nombre del evento.
- ``DeviceIDHash``: identificador de usuario unívoco.
- ``EventTimestamp``: hora del evento.
- ``ExpId``: número de experimento. 246 y 247 son los grupos de control, y 248 es el grupo de prueba.

In [112]:
# Cargar todas las librerias
import pandas as pd 
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np

In [113]:
# Cargar el dataset

data_event=pd.read_csv('/Users/ASUS/OneDrive/Documentos/Tripleten/Proyecto Spring 11/logs_exp_us.csv', sep='\t')

print(data_event)

                      EventName         DeviceIDHash  EventTimestamp  ExpId
0              MainScreenAppear  4575588528974610257      1564029816    246
1              MainScreenAppear  7416695313311560658      1564053102    246
2       PaymentScreenSuccessful  3518123091307005509      1564054127    248
3              CartScreenAppear  3518123091307005509      1564054127    248
4       PaymentScreenSuccessful  6217807653094995999      1564055322    248
...                         ...                  ...             ...    ...
244121         MainScreenAppear  4599628364049201812      1565212345    247
244122         MainScreenAppear  5849806612437486590      1565212439    246
244123         MainScreenAppear  5746969938801999050      1565212483    246
244124         MainScreenAppear  5746969938801999050      1565212498    246
244125       OffersScreenAppear  5746969938801999050      1565212517    246

[244126 rows x 4 columns]


## Prepocesamiento de los datos

De acuerdo con la descripción del dataset, se realiza un preprocesamiento que incluye la eliminación de duplicados, la conversión de los valores de fecha a un formato compatible y la estandarización de los datos. Este proceso permite organizar la información de manera estructurada, garantizar la correcta asignación de tipos de datos y optimizar el rendimiento durante la ejecución del código, lo cual es fundamental para el desarrollo eficiente del análisis.


In [114]:
# Eliminar duplicados
data_event.drop_duplicates(inplace=True)

data_event.info()

<class 'pandas.core.frame.DataFrame'>
Index: 243713 entries, 0 to 244125
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   EventName       243713 non-null  object
 1   DeviceIDHash    243713 non-null  int64 
 2   EventTimestamp  243713 non-null  int64 
 3   ExpId           243713 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 9.3+ MB


In [115]:
#  Renombrar columnas

data_event.rename(columns={'EventName': 'event_name', 'DeviceIDHash': 'device_id_hash', 'EventTimestamp': 'event_timestamp', 'ExpId': 'exp_id'}, inplace=True
)

# Convertir la columna event_timestamp a tipo datetime
data_event['event_timestamp'] = pd.to_datetime(data_event['event_timestamp'] , unit='s')


data_event.info()

<class 'pandas.core.frame.DataFrame'>
Index: 243713 entries, 0 to 244125
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   event_name       243713 non-null  object        
 1   device_id_hash   243713 non-null  int64         
 2   event_timestamp  243713 non-null  datetime64[ns]
 3   exp_id           243713 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 9.3+ MB


In [116]:
# Estandarización de los datos

for col in data_event.select_dtypes(include='object'):
    data_event[col] = data_event[col].str.lower()
    
print(data_event.sample(n=5))

                     event_name       device_id_hash     event_timestamp  \
227273       offersscreenappear   518781617060869985 2019-08-07 12:24:45   
22363   paymentscreensuccessful  9070154092758386922 2019-08-01 13:52:50   
118414         mainscreenappear  6816978929914477854 2019-08-04 10:20:56   
178888         mainscreenappear  8092747482833887138 2019-08-06 04:21:05   
203563       offersscreenappear  2747519232871077339 2019-08-06 16:29:16   

        exp_id  
227273     246  
22363      246  
118414     248  
178888     246  
203563     248  
